# Advanced Scenarios and Optimization

This notebook combines all the previous analysis techniques to perform comprehensive parameter studies and optimization of the CERN ion injector chain. We'll explore combined effects of electron cooling, bunch splitting, and stripper foil placement to find optimal configurations.

## Overview

This advanced example demonstrates:
1. **Multi-parameter optimization**: Testing combinations of all parameters
2. **Scenario comparison**: Direct comparison of realistic operational scenarios
3. **Physics-driven optimization**: Optimizing for different physics goals
4. **Comprehensive analysis**: Understanding parameter interdependencies
5. **Practical recommendations**: Actionable results for operations

Let's dive into comprehensive scenario analysis!

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from pathlib import Path
import sys
from itertools import product
import seaborn as sns

# Add the parent directory to Python path to import injector_model
sys.path.append(str(Path.cwd().parent))
from injector_model import InjectorChain

# Set up matplotlib and seaborn for professional plots
plt.rcParams.update({
    "font.family": "serif",
    "font.size": 11,
    "axes.titlesize": 13,
    "axes.labelsize": 12,
    "legend.fontsize": 10,
})

# Try to use seaborn for enhanced plots, fall back if not available
try:
    sns.set_palette("husl")
    print("✓ Libraries imported successfully (with seaborn)!")
except ImportError:
    print("✓ Libraries imported successfully (without seaborn)!")

## 1. Define Comprehensive Scenario Matrix

Let's define realistic operational scenarios that combine different parameter settings:

In [ ]:
# Define realistic operational scenarios
scenarios_config = {
    "Baseline_Pb_like": {
        "LEIR_bunches": 2,
        "PS_splitting": 2,
        "LEIR_PS_strip": False,
        "account_for_LEIR_ecooling": True,
        "description": "Current Pb-like operation with e-cooling"
    },
    "Conservative_no_ecool": {
        "LEIR_bunches": 2,
        "PS_splitting": 2,
        "LEIR_PS_strip": False,
        "account_for_LEIR_ecooling": False,
        "description": "Conservative without electron cooling"
    },
    "High_splitting_LEIR_PS": {
        "LEIR_bunches": 2,
        "PS_splitting": 4,
        "LEIR_PS_strip": True,
        "account_for_LEIR_ecooling": True,
        "description": "High splitting with LEIR-PS stripping"
    },
    "No_splitting_PS_SPS": {
        "LEIR_bunches": 2,
        "PS_splitting": 1,
        "LEIR_PS_strip": False,
        "account_for_LEIR_ecooling": True,
        "description": "Maximum bunch intensity (no PS splitting)"
    },
    "Optimal_splitting": {
        "LEIR_bunches": 2,
        "PS_splitting": 3,
        "LEIR_PS_strip": False,
        "account_for_LEIR_ecooling": True,
        "description": "Optimized PS splitting with e-cooling"
    },
    "High_current_LEIR_PS": {
        "LEIR_bunches": 2,
        "PS_splitting": 3,
        "LEIR_PS_strip": True,
        "account_for_LEIR_ecooling": True,
        "description": "High current with LEIR-PS stripping"
    }
}

print("Defined Operational Scenarios:")
print("=" * 60)
for name, config in scenarios_config.items():
    print(f"{name}:")
    print(f"  Description: {config['description']}")
    print(f"  Parameters: LEIR_bunches={config['LEIR_bunches']}, "
          f"PS_splitting={config['PS_splitting']}, "
          f"LEIR_PS_strip={config['LEIR_PS_strip']}, "
          f"e-cooling={config['account_for_LEIR_ecooling']}")
    print()

print(f"Total scenarios to analyze: {len(scenarios_config)}")

## 2. Calculate Results for All Scenarios

Let's calculate the performance for all scenarios and all ion species:

In [ ]:
# Calculate results for all scenarios
print("Calculating results for all scenarios...")
print("This will take a few minutes - calculating 6 scenarios for all ion species...")
print()

scenarios_results = {}
scenarios_chains = {}

for scenario_name, config in scenarios_config.items():
    print(f"Calculating scenario: {scenario_name}")
    
    # Create InjectorChain instance
    injector_chain = InjectorChain(
        LEIR_bunches=config['LEIR_bunches'],
        PS_splitting=config['PS_splitting'],
        LEIR_PS_strip=config['LEIR_PS_strip'],
        account_for_LEIR_ecooling=config['account_for_LEIR_ecooling'],
        account_for_PS_rest_gas=True  # Always account for PS rest gas
    )
    
    # Calculate results
    results = injector_chain.calculate_LHC_bunch_intensity_all_ion_species(save_csv=False)
    
    # Store results and injector chain
    scenarios_results[scenario_name] = results
    scenarios_chains[scenario_name] = injector_chain
    
    print(f"  ✓ Completed for {len(results)} ion species")

print(f"\n✅ All scenarios calculated successfully!")
ions = list(scenarios_results[list(scenarios_results.keys())[0]].index)
print(f"Ion species: {ions}")

## 3. Comprehensive Performance Comparison

Let's create comprehensive visualizations comparing all scenarios:

In [ ]:
# Create comprehensive performance comparison
fig = plt.figure(figsize=(20, 16))

# Define a consistent color palette for scenarios
scenario_colors = {
    'Baseline_Pb_like': '#1f77b4',
    'Conservative_no_ecool': '#ff7f0e', 
    'High_splitting_LEIR_PS': '#2ca02c',
    'No_splitting_PS_SPS': '#d62728',
    'Optimal_splitting': '#9467bd',
    'High_current_LEIR_PS': '#8c564b'
}

# Subplot 1: LHC ions per bunch comparison
ax1 = plt.subplot(2, 3, 1)
x_pos = np.arange(len(ions))
width = 0.13
for i, (scenario_name, results) in enumerate(scenarios_results.items()):
    offset = (i - len(scenarios_results)/2 + 0.5) * width
    ax1.bar(x_pos + offset, results['LHC_ionsPerBunch'], width, 
           label=scenario_name.replace('_', ' '), 
           color=scenario_colors[scenario_name], alpha=0.8)

ax1.set_xlabel('Ion Species')
ax1.set_ylabel('LHC Ions per Bunch')
ax1.set_title('Individual Bunch Intensity')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(ions, rotation=45)
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# Subplot 2: Total ion current (all bunches)
ax2 = plt.subplot(2, 3, 2)
for i, (scenario_name, results) in enumerate(scenarios_results.items()):
    config = scenarios_config[scenario_name]
    total_ions = (results['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                 results['LEIR_bunches'] * results['LEIR_numberofPulses'])
    offset = (i - len(scenarios_results)/2 + 0.5) * width
    ax2.bar(x_pos + offset, total_ions, width, 
           label=scenario_name.replace('_', ' '), 
           color=scenario_colors[scenario_name], alpha=0.8)

ax2.set_xlabel('Ion Species')
ax2.set_ylabel('Total Ion Current')
ax2.set_title('Total Beam Current (All Bunches)')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(ions, rotation=45)
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Subplot 3: Total nucleons
ax3 = plt.subplot(2, 3, 3)
for i, (scenario_name, results) in enumerate(scenarios_results.items()):
    config = scenarios_config[scenario_name]
    total_nucleons = (results['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                     results['LEIR_bunches'] * results['LEIR_numberofPulses'] * 
                     results['massNumber'])
    offset = (i - len(scenarios_results)/2 + 0.5) * width
    ax3.bar(x_pos + offset, total_nucleons, width, 
           label=scenario_name.replace('_', ' '), 
           color=scenario_colors[scenario_name], alpha=0.8)

ax3.set_xlabel('Ion Species')
ax3.set_ylabel('Total Nucleons')
ax3.set_title('Total Nucleon Current')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(ions, rotation=45)
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3)

# Subplot 4: Number of LEIR injections
ax4 = plt.subplot(2, 3, 4)
for i, (scenario_name, results) in enumerate(scenarios_results.items()):
    offset = (i - len(scenarios_results)/2 + 0.5) * width
    ax4.bar(x_pos + offset, results['LEIR_numberofPulses'], width, 
           label=scenario_name.replace('_', ' '), 
           color=scenario_colors[scenario_name], alpha=0.8)

ax4.set_xlabel('Ion Species')
ax4.set_ylabel('Number of LEIR Injections')
ax4.set_title('LEIR Injection Capability')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(ions, rotation=45)
ax4.grid(True, alpha=0.3)

# Subplot 5: Total number of LHC bunches
ax5 = plt.subplot(2, 3, 5)
for i, (scenario_name, results) in enumerate(scenarios_results.items()):
    config = scenarios_config[scenario_name]
    total_bunches = (results['LEIR_bunches'] * config['PS_splitting'] * 
                    results['LEIR_numberofPulses'])
    offset = (i - len(scenarios_results)/2 + 0.5) * width
    ax5.bar(x_pos + offset, total_bunches, width, 
           label=scenario_name.replace('_', ' '), 
           color=scenario_colors[scenario_name], alpha=0.8)

ax5.set_xlabel('Ion Species')
ax5.set_ylabel('Total LHC Bunches')
ax5.set_title('Total Bunch Count in LHC')
ax5.set_xticks(x_pos)
ax5.set_xticklabels(ions, rotation=45)
ax5.grid(True, alpha=0.3)

# Subplot 6: Space charge utilization in SPS
ax6 = plt.subplot(2, 3, 6)
for i, (scenario_name, results) in enumerate(scenarios_results.items()):
    sc_utilization = results['SPS_maxIntensity'] / results['SPS_space_charge_limit']
    offset = (i - len(scenarios_results)/2 + 0.5) * width
    ax6.bar(x_pos + offset, sc_utilization, width, 
           label=scenario_name.replace('_', ' '), 
           color=scenario_colors[scenario_name], alpha=0.8)

ax6.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='SC limit')
ax6.set_xlabel('Ion Species')
ax6.set_ylabel('SPS Intensity / SC Limit')
ax6.set_title('SPS Space Charge Utilization')
ax6.set_xticks(x_pos)
ax6.set_xticklabels(ions, rotation=45)
ax6.set_yscale('log')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Comprehensive scenario comparison completed!")
print("   Each subplot shows different performance metrics across all scenarios")

## 3.5. Comparison with Reference Values

Let's also create a comparison bar chart with selected scenarios in the calculation script style:

In [ ]:
# Load reference data for comparison
data_folder = Path.cwd().parent / 'data'
ref_Table_SPS = pd.read_csv(data_folder / 'test_and_benchmark_data' / 'SPS_final_intensities_WG5_and_Hannes.csv', index_col=0)

# Set up plotting parameters to match calculation script style
SMALL_SIZE = 11
MEDIUM_SIZE = 12
BIGGER_SIZE = 13
plt.rcParams["font.family"] = "serif"
plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE)
plt.rc('axes', labelsize=BIGGER_SIZE)
plt.rc('xtick', labelsize=MEDIUM_SIZE)
plt.rc('ytick', labelsize=MEDIUM_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)

# Select key scenarios for comparison (to avoid overcrowding)
key_scenarios = ['Baseline_Pb_like', 'Conservative_no_ecool', 'High_performance', 'Optimized_light_ions']
scenario_colors_compact = ['blue', 'gray', 'gold', 'limegreen']

# Define bar width for bar plot
bar_width = 0.15
x = np.arange(len(ions))

# Create comparison plot: Nucleons per bunch for selected scenarios
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
bar1 = ax.bar(x, ref_Table_SPS['WG5 Intensity']*scenarios_results['Baseline_Pb_like']['massNumber'], 
              bar_width, color='red', label='WG5')
for i, scenario in enumerate(key_scenarios):
    offset = (i + 1) * bar_width
    bar = ax.bar(x + offset, scenarios_results[scenario]['LHC_ionsPerBunch']*scenarios_results[scenario]['massNumber'], 
                 bar_width, color=scenario_colors_compact[i], 
                 label=scenario.replace('_', ' '))

ax.set_xticks(x + 2.5*bar_width)
ax.set_xticklabels(ions)
ax.set_ylabel("Nucleons per bunch")
ax.legend(fontsize=10)
fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.show()

print("Comparison plot shows nucleons per bunch for key optimization scenarios")
print("Red: WG5 reference values")
for i, scenario in enumerate(key_scenarios):
    print(f"{scenario_colors_compact[i].capitalize()}: {scenario.replace('_', ' ')}")

## 4. Optimization Analysis: Find Best Scenario for Each Ion

Let's determine which scenario is optimal for each ion species and optimization objective:

In [ ]:
# Create optimization analysis
optimization_data = []

for ion in ions:
    ion_data = {'Ion': ion}
    
    # Extract metrics for this ion from all scenarios
    metrics = {}
    for scenario_name, results in scenarios_results.items():
        config = scenarios_config[scenario_name]
        row = results.loc[ion]
        
        metrics[scenario_name] = {
            'ions_per_bunch': row['LHC_ionsPerBunch'],
            'charges_per_bunch': row['LHC_chargesPerBunch'],
            'total_ions': (row['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                          row['LEIR_bunches'] * row['LEIR_numberofPulses']),
            'total_nucleons': (row['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                             row['LEIR_bunches'] * row['LEIR_numberofPulses'] * 
                             row['massNumber']),
            'total_bunches': (row['LEIR_bunches'] * config['PS_splitting'] * 
                            row['LEIR_numberofPulses']),
            'sps_sc_utilization': row['SPS_maxIntensity'] / row['SPS_space_charge_limit']
        }
    
    # Find best scenario for each objective
    best_bunch_intensity = max(metrics, key=lambda x: metrics[x]['ions_per_bunch'])
    best_total_ions = max(metrics, key=lambda x: metrics[x]['total_ions'])
    best_nucleons = max(metrics, key=lambda x: metrics[x]['total_nucleons'])
    best_bunch_count = max(metrics, key=lambda x: metrics[x]['total_bunches'])
    
    # Store results
    ion_data.update({
        'Mass': scenarios_results[list(scenarios_results.keys())[0]].loc[ion]['massNumber'],
        'Atomic': scenarios_results[list(scenarios_results.keys())[0]].loc[ion]['atomicNumber'],
        'Best_for_bunch_intensity': best_bunch_intensity,
        'Best_for_total_ions': best_total_ions,
        'Best_for_nucleons': best_nucleons,
        'Best_for_bunch_count': best_bunch_count,
        'Max_bunch_intensity': metrics[best_bunch_intensity]['ions_per_bunch'],
        'Max_total_ions': metrics[best_total_ions]['total_ions'],
        'Max_nucleons': metrics[best_nucleons]['total_nucleons'],
        'Max_bunch_count': metrics[best_bunch_count]['total_bunches']
    })
    
    optimization_data.append(ion_data)

opt_df = pd.DataFrame(optimization_data)

# Display optimization results table
print("Optimization Results: Best Scenario for Each Objective")
print("=" * 80)
display_df = opt_df[['Ion', 'Mass', 'Atomic', 'Best_for_bunch_intensity', 
                    'Best_for_total_ions', 'Best_for_nucleons']].copy()

# Shorten scenario names for display
name_mapping = {
    'Baseline_Pb_like': 'Baseline',
    'Conservative_no_ecool': 'No_ecool',
    'High_splitting_LEIR_PS': 'Hi_split_LP',
    'No_splitting_PS_SPS': 'No_split',
    'Optimal_splitting': 'Opt_split',
    'High_current_LEIR_PS': 'Hi_curr_LP'
}

for col in ['Best_for_bunch_intensity', 'Best_for_total_ions', 'Best_for_nucleons']:
    display_df[col] = display_df[col].map(name_mapping)

display_df.columns = ['Ion', 'A', 'Z', 'Best: Bunch Int.', 'Best: Total Ions', 'Best: Nucleons']
print(display_df.to_string(index=False))

# Count frequency of each scenario being optimal
print("\n\nScenario Optimization Frequency:")
print("=" * 50)
objectives = ['Best_for_bunch_intensity', 'Best_for_total_ions', 'Best_for_nucleons']
objective_names = ['Bunch Intensity', 'Total Ions', 'Total Nucleons']

for obj, obj_name in zip(objectives, objective_names):
    counts = opt_df[obj].value_counts()
    print(f"\n{obj_name}:")
    for scenario, count in counts.items():
        percentage = (count / len(ions)) * 100
        short_name = name_mapping.get(scenario, scenario)
        print(f"  {short_name:<12}: {count} ions ({percentage:.1f}%)")

## 5. Performance Heatmaps

Let's create heatmaps to visualize the relative performance of each scenario:

In [ ]:
# Create performance heatmaps
def create_performance_matrix(metric_name, scenarios_results, scenarios_config, normalize=True):
    """Create a performance matrix for heatmap visualization"""
    matrix_data = []
    
    for ion in ions:
        row_data = []
        for scenario_name in scenarios_config.keys():
            results = scenarios_results[scenario_name]
            config = scenarios_config[scenario_name]
            ion_row = results.loc[ion]
            
            if metric_name == 'ions_per_bunch':
                value = ion_row['LHC_ionsPerBunch']
            elif metric_name == 'total_ions':
                value = (ion_row['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                        ion_row['LEIR_bunches'] * ion_row['LEIR_numberofPulses'])
            elif metric_name == 'total_nucleons':
                value = (ion_row['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                        ion_row['LEIR_bunches'] * ion_row['LEIR_numberofPulses'] * 
                        ion_row['massNumber'])
            elif metric_name == 'sps_sc_ratio':
                value = ion_row['SPS_maxIntensity'] / ion_row['SPS_space_charge_limit']
            
            row_data.append(value)
        
        # Normalize to relative performance if requested
        if normalize:
            max_val = max(row_data)
            if max_val > 0:
                row_data = [val/max_val for val in row_data]
        
        matrix_data.append(row_data)
    
    return np.array(matrix_data)

# Create heatmaps for different metrics
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

scenario_names_short = [name_mapping[name] for name in scenarios_config.keys()]

# Heatmap 1: Relative bunch intensity performance
matrix1 = create_performance_matrix('ions_per_bunch', scenarios_results, scenarios_config)
im1 = ax1.imshow(matrix1, cmap='RdYlGn', aspect='auto', interpolation='nearest')
ax1.set_xlabel('Scenario')
ax1.set_ylabel('Ion Species')
ax1.set_title('Relative Bunch Intensity Performance')
ax1.set_xticks(range(len(scenario_names_short)))
ax1.set_xticklabels(scenario_names_short, rotation=45)
ax1.set_yticks(range(len(ions)))
ax1.set_yticklabels(ions)
plt.colorbar(im1, ax=ax1, label='Relative Performance')

# Heatmap 2: Relative total ion current performance
matrix2 = create_performance_matrix('total_ions', scenarios_results, scenarios_config)
im2 = ax2.imshow(matrix2, cmap='RdYlGn', aspect='auto', interpolation='nearest')
ax2.set_xlabel('Scenario')
ax2.set_ylabel('Ion Species')
ax2.set_title('Relative Total Ion Current Performance')
ax2.set_xticks(range(len(scenario_names_short)))
ax2.set_xticklabels(scenario_names_short, rotation=45)
ax2.set_yticks(range(len(ions)))
ax2.set_yticklabels(ions)
plt.colorbar(im2, ax=ax2, label='Relative Performance')

# Heatmap 3: Relative nucleon production performance
matrix3 = create_performance_matrix('total_nucleons', scenarios_results, scenarios_config)
im3 = ax3.imshow(matrix3, cmap='RdYlGn', aspect='auto', interpolation='nearest')
ax3.set_xlabel('Scenario')
ax3.set_ylabel('Ion Species')
ax3.set_title('Relative Nucleon Production Performance')
ax3.set_xticks(range(len(scenario_names_short)))
ax3.set_xticklabels(scenario_names_short, rotation=45)
ax3.set_yticks(range(len(ions)))
ax3.set_yticklabels(ions)
plt.colorbar(im3, ax=ax3, label='Relative Performance')

# Heatmap 4: SPS space charge utilization
matrix4 = create_performance_matrix('sps_sc_ratio', scenarios_results, scenarios_config, normalize=False)
im4 = ax4.imshow(matrix4, cmap='RdYlBu_r', aspect='auto', interpolation='nearest')
ax4.set_xlabel('Scenario')
ax4.set_ylabel('Ion Species')
ax4.set_title('SPS Space Charge Utilization')
ax4.set_xticks(range(len(scenario_names_short)))
ax4.set_xticklabels(scenario_names_short, rotation=45)
ax4.set_yticks(range(len(ions)))
ax4.set_yticklabels(ions)
plt.colorbar(im4, ax=ax4, label='Intensity / SC Limit')

plt.tight_layout()
plt.show()

print("🌡️ Performance heatmaps:")
print("   • Green = Best performance, Red = Worst performance")
print("   • Each row is normalized to show relative performance for that ion")
print("   • SPS SC utilization: Blue = Below limit, Red = Above limit")

## 6. Physics-Based Recommendations

Let's create physics-based recommendations for different operational goals:

In [ ]:
# Create physics-based recommendations
recommendations_analysis = []

for ion in ions:
    ion_rec = {'Ion': ion}
    
    # Get data for this ion from all scenarios
    ion_metrics = {}
    for scenario_name, results in scenarios_results.items():
        config = scenarios_config[scenario_name]
        row = results.loc[ion]
        
        ion_metrics[scenario_name] = {
            'bunch_intensity': row['LHC_ionsPerBunch'],
            'total_current': (row['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                            row['LEIR_bunches'] * row['LEIR_numberofPulses']),
            'nucleon_production': (row['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                                 row['LEIR_bunches'] * row['LEIR_numberofPulses'] * 
                                 row['massNumber']),
            'sps_sc_safety': 1.0 - (row['SPS_maxIntensity'] / row['SPS_space_charge_limit']),
            'total_bunches': (row['LEIR_bunches'] * config['PS_splitting'] * 
                            row['LEIR_numberofPulses'])
        }
    
    # Physics-based recommendations
    
    # 1. For peak luminosity (high bunch intensity)
    best_peak_lumi = max(ion_metrics, key=lambda x: ion_metrics[x]['bunch_intensity'])
    peak_lumi_value = ion_metrics[best_peak_lumi]['bunch_intensity']
    
    # 2. For integrated luminosity (high total current)
    best_int_lumi = max(ion_metrics, key=lambda x: ion_metrics[x]['total_current'])
    int_lumi_value = ion_metrics[best_int_lumi]['total_current']
    
    # 3. For nucleon-nucleon collisions (ALICE3 goal)
    best_nn_collisions = max(ion_metrics, key=lambda x: ion_metrics[x]['nucleon_production'])
    nn_value = ion_metrics[best_nn_collisions]['nucleon_production']
    
    # 4. For operational safety (low space charge)
    best_safety = max(ion_metrics, key=lambda x: ion_metrics[x]['sps_sc_safety'])
    safety_value = ion_metrics[best_safety]['sps_sc_safety']
    
    # 5. Overall balanced recommendation (weighted score)
    def balanced_score(scenario):
        metrics = ion_metrics[scenario]
        # Normalize each metric to 0-1 scale across scenarios
        bunch_norm = metrics['bunch_intensity'] / max(ion_metrics[s]['bunch_intensity'] for s in ion_metrics)
        total_norm = metrics['total_current'] / max(ion_metrics[s]['total_current'] for s in ion_metrics)
        nucleon_norm = metrics['nucleon_production'] / max(ion_metrics[s]['nucleon_production'] for s in ion_metrics)
        safety_norm = max(0, metrics['sps_sc_safety']) / max(max(0, ion_metrics[s]['sps_sc_safety']) for s in ion_metrics)
        
        # Weighted combination (adjust weights as needed)
        return 0.3*bunch_norm + 0.3*total_norm + 0.3*nucleon_norm + 0.1*safety_norm
    
    best_balanced = max(ion_metrics, key=balanced_score)
    balanced_score_value = balanced_score(best_balanced)
    
    ion_rec.update({
        'Peak_Luminosity_Scenario': best_peak_lumi,
        'Integrated_Luminosity_Scenario': best_int_lumi,
        'ALICE3_NN_Scenario': best_nn_collisions,
        'Operational_Safety_Scenario': best_safety,
        'Balanced_Recommendation': best_balanced,
        'Peak_Lumi_Value': peak_lumi_value,
        'Int_Lumi_Value': int_lumi_value,
        'NN_Value': nn_value,
        'Balanced_Score': balanced_score_value
    })
    
    recommendations_analysis.append(ion_rec)

rec_df = pd.DataFrame(recommendations_analysis)

# Create summary table with shortened names
summary_rec = rec_df[['Ion', 'Peak_Luminosity_Scenario', 'Integrated_Luminosity_Scenario', 
                     'ALICE3_NN_Scenario', 'Balanced_Recommendation']].copy()

for col in ['Peak_Luminosity_Scenario', 'Integrated_Luminosity_Scenario', 
           'ALICE3_NN_Scenario', 'Balanced_Recommendation']:
    summary_rec[col] = summary_rec[col].map(name_mapping)

summary_rec.columns = ['Ion', 'Peak Lumi', 'Integrated Lumi', 'ALICE3 (NN)', 'Balanced']

print("Physics-Based Scenario Recommendations:")
print("=" * 70)
print(summary_rec.to_string(index=False))

# Count most common recommendations
print("\n\nMost Common Recommendations by Physics Goal:")
print("=" * 55)
goals = ['Peak_Luminosity_Scenario', 'Integrated_Luminosity_Scenario', 
         'ALICE3_NN_Scenario', 'Balanced_Recommendation']
goal_names = ['Peak Luminosity', 'Integrated Luminosity', 'ALICE3 (N-N)', 'Balanced']

for goal, goal_name in zip(goals, goal_names):
    counts = rec_df[goal].value_counts()
    print(f"\n{goal_name}:")
    for scenario, count in counts.items():
        percentage = (count / len(ions)) * 100
        short_name = name_mapping.get(scenario, scenario)
        print(f"  {short_name:<12}: {count} ions ({percentage:.1f}%)")

# Find the most universally recommended scenario
all_recommendations = []
for goal in goals:
    all_recommendations.extend(rec_df[goal].tolist())

most_common_overall = pd.Series(all_recommendations).value_counts().index[0]
print(f"\n🎯 Most universally recommended: {name_mapping[most_common_overall]}")
print(f"   ({scenarios_config[most_common_overall]['description']})")

## 7. Create Final Summary and Actionable Recommendations

Let's create a comprehensive summary with clear operational recommendations:

In [ ]:
# Create final comprehensive summary
print("🎯 COMPREHENSIVE INJECTOR CHAIN OPTIMIZATION SUMMARY")
print("=" * 65)

# Overall performance statistics
print("\n1. OVERALL PERFORMANCE STATISTICS:")
print("-" * 40)

for scenario_name, config in scenarios_config.items():
    results = scenarios_results[scenario_name]
    short_name = name_mapping[scenario_name]
    
    # Calculate average performance metrics
    avg_bunch_intensity = results['LHC_ionsPerBunch'].mean()
    avg_total_ions = (results['LHC_ionsPerBunch'] * config['PS_splitting'] * 
                     results['LEIR_bunches'] * results['LEIR_numberofPulses']).mean()
    avg_sps_sc = (results['SPS_maxIntensity'] / results['SPS_space_charge_limit']).mean()
    
    print(f"{short_name:<12}: Bunch={avg_bunch_intensity:.1e}, Total={avg_total_ions:.1e}, SC={avg_sps_sc:.2f}")

# Ion-specific recommendations
print("\n\n2. ION-SPECIFIC OPERATIONAL RECOMMENDATIONS:")
print("-" * 50)

for _, row in rec_df.iterrows():
    ion = row['Ion']
    balanced = name_mapping[row['Balanced_Recommendation']]
    peak_lumi = name_mapping[row['Peak_Luminosity_Scenario']]
    alice3 = name_mapping[row['ALICE3_NN_Scenario']]
    
    print(f"{ion}:")
    print(f"  • General purpose: {balanced}")
    if peak_lumi != balanced:
        print(f"  • Peak luminosity: {peak_lumi}")
    if alice3 != balanced:
        print(f"  • ALICE3 (N-N): {alice3}")
    
    # Add specific insights
    mass = scenarios_results[list(scenarios_results.keys())[0]].loc[ion]['massNumber']
    if mass < 40:
        print(f"  • Light ion: Benefits strongly from electron cooling")
    elif mass > 100:
        print(f"  • Heavy ion: Consider operational stability")
    print()

# Key physics insights
print("\n3. KEY PHYSICS INSIGHTS:")
print("-" * 30)
print("• Electron cooling: Essential for light ions, beneficial for all")
print("• PS splitting: Trade-off between bunch intensity and total current")
print("• Stripper placement: Ion-dependent optimization needed")
print("• Space charge: Most limiting factor in SPS for high intensities")
print("• LEIR injections: Bottleneck for many scenarios")

# Operational recommendations
print("\n\n4. OPERATIONAL RECOMMENDATIONS:")
print("-" * 35)

# Find most common scenarios
all_balanced_recs = rec_df['Balanced_Recommendation'].value_counts()
most_versatile = all_balanced_recs.index[0]
second_most = all_balanced_recs.index[1] if len(all_balanced_recs) > 1 else most_versatile

print(f"🥇 PRIMARY RECOMMENDATION: {name_mapping[most_versatile]}")
print(f"   - {scenarios_config[most_versatile]['description']}")
print(f"   - Optimal for {all_balanced_recs[most_versatile]}/{len(ions)} ions")
print(f"   - Parameters: LEIR_bunches={scenarios_config[most_versatile]['LEIR_bunches']}, "
      f"PS_splitting={scenarios_config[most_versatile]['PS_splitting']}, "
      f"LEIR_PS_strip={scenarios_config[most_versatile]['LEIR_PS_strip']}")

print(f"\n🥈 SECONDARY RECOMMENDATION: {name_mapping[second_most]}")
print(f"   - {scenarios_config[second_most]['description']}")
print(f"   - Alternative for {all_balanced_recs[second_most]}/{len(ions)} ions")

# Risk assessment
print("\n\n5. RISK ASSESSMENT:")
print("-" * 20)

# Count ions that hit space charge limits
high_risk_ions = 0
for scenario_name, results in scenarios_results.items():
    if scenario_name == most_versatile:
        sc_ratios = results['SPS_maxIntensity'] / results['SPS_space_charge_limit']
        high_risk_ions = (sc_ratios >= 0.9).sum()
        break

print(f"• Space charge risk: {high_risk_ions}/{len(ions)} ions operate near SPS limits")
print(f"• Electron cooling: Required for competitive intensities")
print(f"• Operational complexity: Moderate (standard parameters)")
print(f"• Performance margin: Built-in safety factors in calculations")

# Future studies
print("\n\n6. RECOMMENDED FOLLOW-UP STUDIES:")
print("-" * 38)
print("• Detailed space charge simulations for high-risk ions")
print("• Electron cooling optimization studies")
print("• Beam dynamics studies for new parameter ranges")
print("• Operational procedures for scenario switching")
print("• Luminosity performance validation with colliding beams")

print("\n" + "=" * 65)
print("🚀 READY FOR LIGHT ION PHYSICS AT THE LHC!")
print("=" * 65)

## Summary

This comprehensive notebook has demonstrated advanced analysis techniques for optimizing the CERN ion injector chain:

### 🔬 **Analysis Techniques Demonstrated:**

1. **Multi-Parameter Optimization**: Systematic exploration of parameter combinations
2. **Scenario-Based Analysis**: Realistic operational scenario comparison
3. **Performance Heatmaps**: Visual identification of optimal configurations
4. **Physics-Goal Optimization**: Tailored recommendations for different physics objectives
5. **Risk Assessment**: Evaluation of operational safety and space charge margins

### 🎯 **Key Scientific Insights:**

1. **No Universal Optimum**: Different ion species and physics goals require different optimization strategies

2. **Critical Parameter Interdependencies**:
   - Electron cooling enables higher intensities but increases complexity
   - PS splitting trades individual bunch intensity for total current
   - Stripper placement affects space charge distribution across accelerators

3. **Space Charge as Limiting Factor**: SPS space charge limits are the primary constraint for most high-intensity scenarios

4. **Light Ion Advantages**: Lighter ions generally achieve higher intensities and benefit more from optimization

### 🛠️ **Practical Outcomes:**

1. **Clear Operational Recommendations**: Ion-specific and physics-goal-specific parameter choices

2. **Risk Assessment Framework**: Understanding of operational margins and safety factors

3. **Performance Prediction**: Quantitative predictions for different operational scenarios

4. **Future Study Roadmap**: Identification of critical areas needing further investigation

### 🚀 **Impact for ALICE3 and Future Light Ion Physics:**

This analysis provides the quantitative foundation for:
- **Optimizing injector chain parameters** for maximum nucleon-nucleon luminosity
- **Planning operational strategies** for different ion species
- **Assessing feasibility** of ALICE3 luminosity requirements
- **Guiding future upgrades** and operational procedures

The combination of sophisticated modeling, comprehensive parameter studies, and physics-based optimization demonstrated here represents a complete framework for **scientific decision-making in accelerator physics**.

### 🔧 **Tools and Techniques You've Learned:**

- Multi-dimensional parameter optimization
- Performance heatmap visualization
- Scenario-based comparative analysis
- Physics-goal-driven optimization
- Risk assessment methodologies
- Comprehensive scientific reporting

This completes our comprehensive exploration of the `injector_model` package capabilities! 🎉